1. Import the Module
At the top of app.py, add this import:

In [ ]:
import plot_utils as viz  # Importing your new file

2. Add the Visualization Logic
Replace the existing "Prediction Engine" section (inside with tab1:) with this updated code block. This connects the button click to your new calculation engine and displays the charts.

In [ ]:
# --- INSIDE app.py (Replace the existing button logic) ---

with tab1:
    st.write("### Prediction Engine")

    if st.button("Run Risk Analysis"):
        # 1. Prepare the single row of data
        # We convert the selected Series to a DataFrame (1 row)
        row_data = selected_row.to_frame().T

        # 2. CALL THE ENGINE (from plot_utils.py)
        # This function handles all the SHAP math, scaling, and aggregation
        df_pillars, df_impacts, prob_success, error = viz.calculate_risk_drivers(
            model, explainer, taxonomy, row_data
        )

        if error:
            st.error(f"Analysis Failed: {error}")
        else:
            # --- 3. DISPLAY RESULTS ---

            # Top Row: Gauge and Bar Chart
            col1, col2 = st.columns([1, 2])

            with col1:
                st.subheader("Success Probability")
                # Generate Gauge
                fig_gauge = viz.plot_success_gauge(prob_success)
                st.plotly_chart(fig_gauge, use_container_width=True)

                # Add a text interpretation
                if prob_success > 0.5:
                    st.success(f"Likely to Succeed ({prob_success:.1%})")
                else:
                    st.error(f"High Risk of Failure ({prob_success:.1%})")

            with col2:
                st.subheader("Key Drivers (Pillars)")
                # Generate Bar Chart
                fig_bar = viz.plot_impact_bar(df_pillars)
                st.plotly_chart(fig_bar, use_container_width=True)

            # Bottom Row: Treemap (Deep Dive)
            st.subheader("Deep Dive: Specific Risk Factors")
            st.write("This chart shows exactly which features are driving the risk within each pillar.")
            # Generate Treemap
            fig_tree = viz.plot_treemap(df_impacts)
            st.plotly_chart(fig_tree, use_container_width=True)


**Summary of what this does:**
1.  **User clicks button.**
2.  `app.py` sends the data to `viz.calculate_risk_drivers`.
3.  Your `plot_utils.py` runs SHAP, calculates the probability, and groups the features.
4.  `app.py` receives the clean dataframes (`df_pillars`, `df_impacts`).
5.  `app.py` calls your plotting functions (`viz.plot_success_gauge`, etc.) to draw the charts.

This keeps the `app.py` clean and easy to read, while all the complex math stays safely inside `plot_utils.py`.